In [35]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import time
import random
import cv2
import pandas as pd
from keras import layers, models, optimizers, losses, metrics, Sequential
from keras.layers import Dense, Conv2D, Flatten , Dropout, MaxPooling2D

In [36]:
# Generate our dataset for suits using the labeled images in the suit_images directory
# We will use the keras ImageDataGenerator to generate our dataset

suit_train = tf.keras.utils.image_dataset_from_directory("suit_images", labels="inferred", label_mode="categorical", color_mode="grayscale", batch_size=32, image_size=(100, 100), shuffle=True, seed=123, interpolation="bilinear", follow_links=False, validation_split=0.2, subset="training")
suit_validate = tf.keras.utils.image_dataset_from_directory("suit_images", labels="inferred", label_mode="categorical", color_mode="grayscale", batch_size=32, image_size=(100, 100), shuffle=True, seed=123, validation_split=0.2, subset="validation", interpolation="bilinear", follow_links=False)

print(suit_train.class_names)
print("Sizes: ", suit_train.cardinality(), suit_validate.cardinality())

print("Suit Train Shape: ", suit_train.element_spec)




Found 329 files belonging to 4 classes.
Using 264 files for training.
Found 329 files belonging to 4 classes.
Using 65 files for validation.
['clubs', 'diamonds', 'hearts', 'spades']
Sizes:  tf.Tensor(9, shape=(), dtype=int64) tf.Tensor(3, shape=(), dtype=int64)
Suit Train Shape:  (TensorSpec(shape=(None, 100, 100, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))


In [37]:
# Lets make a network
# We will use a convolutional neural network

# First we will define our model
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(100,100,1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(4, activation="softmax"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 100, 100, 32)      320       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 50, 50, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 25, 25, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 25, 25, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 12, 12, 64)      

In [39]:
# Train the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(suit_train, epochs=10, validation_data=suit_validate)

Epoch 1/10
9/9 [==============================] - 2s 127ms/step - loss: 0.0881 - accuracy: 0.9811 - val_loss: 0.2893 - val_accuracy: 0.9077
Epoch 2/10
9/9 [==============================] - 1s 120ms/step - loss: 0.0179 - accuracy: 1.0000 - val_loss: 0.2543 - val_accuracy: 0.8923
Epoch 3/10
9/9 [==============================] - 1s 108ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.3321 - val_accuracy: 0.8769
Epoch 4/10
9/9 [==============================] - 1s 104ms/step - loss: 0.0254 - accuracy: 0.9886 - val_loss: 0.8124 - val_accuracy: 0.8154
Epoch 5/10
9/9 [==============================] - 1s 104ms/step - loss: 0.0151 - accuracy: 0.9962 - val_loss: 0.2131 - val_accuracy: 0.9077
Epoch 6/10
9/9 [==============================] - 1s 105ms/step - loss: 0.0067 - accuracy: 0.9962 - val_loss: 0.3336 - val_accuracy: 0.9077
Epoch 7/10
9/9 [==============================] - 1s 111ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.3016 - val_accuracy: 0.9385
Epoch 8/10
9/9 [====

In [40]:
# Save the model
model.save("suit_model.h5")

In [41]:
# Lets test the model

# Load the model in
model = tf.keras.models.load_model("suit_model.h5")

# Load the test image
test_image = cv2.imread("suit_images/hearts/hearts_10_suit_1680993207.517387.png", cv2.IMREAD_GRAYSCALE)
test_image = cv2.resize(test_image, (100, 100))
test_image = np.array(test_image)
test_image = test_image.reshape(1, 100, 100, 1)

# Make a prediction
prediction = model.predict(test_image)
print(prediction)
class_index = np.argmax(prediction)
print(class_index)
print(suit_train.class_names[class_index])



[[7.0409200e-08 4.4852359e-06 9.9998927e-01 6.0837583e-06]]
2
hearts


In [34]:
# Run through all of the images in the suit_images/hearts directory
# and make a prediction for each one
# Then print out the accuracy

# Get a list of all of the files in the directory using pathlib
path = pathlib.Path("suit_images/diamonds")

# Get a list of all of the files in the directory
files = list(path.glob("*.png"))

# Now lets sort the files by the time they were created
files.sort(key=lambda x: x.stat().st_ctime)

for file in files:
    # Now lets make a prediction for each image
    test_image = cv2.imread(file.as_posix(), cv2.IMREAD_GRAYSCALE)
    test_image = np.array(test_image)
    test_image = test_image.reshape(1, 100, 100, 1)

    # Make a prediction
    prediction = model.predict(test_image)
    class_index = np.argmax(prediction)
    c = suit_train.class_names[class_index]

    if c == "diamonds":
        pass
    else:
        print("Incorrect")
        cv2.imshow("Image", cv2.imread(file.as_posix(), cv2.IMREAD_GRAYSCALE))
        cv2.waitKey(0)


